In [1]:
# mandatory imports
from xcube.core.store import find_data_store_extensions
from xcube.core.store import get_data_store_params_schema
from xcube.core.store import new_data_store

# Utilities for notebook visualization
import shapely.geometry
import IPython.display
import pandas as pd
from IPython.display import JSON
import matplotlib.pyplot as plt
import math

In [2]:
store = new_data_store('sentinelhub', num_retries=400)

In [3]:
store.get_open_data_params_schema('S2L2A')

In [4]:
mc_def = pd.read_csv("../3_eventcoord4minicube.csv", delimiter="\t")
mc_def

,Longitude,Latitude,class
mc1,30,57,7
mc2,25,55,3
mc3,10,37,1
mc4,20,50,0
mc5,30,40,0


In [5]:
r_earth = 6378137
spatial_res_m = 20
spatial_res = 360 * spatial_res_m / (2 * math.pi * r_earth)
spatial_res

0.00017966305682390427

In [6]:
spatial_res = 0.00018
image_size = 256

In [7]:
regions = []
for row in mc_def.iterrows():
    name, (center_lon, center_lat, _) = row
    spatial_size = spatial_res * image_size / 2
    bbox = (center_lon - spatial_size,
            center_lat - spatial_size,
            center_lon + spatial_size,
            center_lat + spatial_size)
    regions.append((name, (center_lon, center_lat), bbox))

In [8]:
polygon = shapely.geometry.MultiPolygon([shapely.geometry.box(*bbox) 
                                        for _, _, bbox in regions])
IPython.display.GeoJSON(polygon.__geo_interface__)

<IPython.display.GeoJSON object>

In [9]:
for name, center, bbox in regions:
    mc = store.open_data('S2L2A', 
                         variable_names=['B02', 'B03', 'B04', 'B06', 'B8A',  'B11', 'SCL'], 
                         bbox=bbox, 
                         spatial_res=spatial_res, 
                         time_range=('2016-01-01','2022-08-01'), 
                         time_period=None)
    mc.to_zarr(f"s2_l2a_{name}_x{center[0]}_y{center[1]}.zarr")

D:\Miniconda3\envs\xcube\lib\site-packages\pandas\core\tools\timedeltas.py:148: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version.
  return _coerce_scalar_to_timedelta_type(arg, unit=unit, errors=errors)
D:\Miniconda3\envs\xcube\lib\site-packages\pandas\core\tools\timedeltas.py:148: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version.
  return _coerce_scalar_to_timedelta_type(arg, unit=unit, errors=errors)
D:\Miniconda3\envs\xcube\lib\site-packages\pandas\core\tools\timedeltas.py:148: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version.
  return _coerce_scalar_to_timedelta_type(arg, unit=unit, errors=errors)


KeyboardInterrupt: 